# SVM 
비선형 최적화 기법으로 Vapnik 에 제한한 학습이론에 기반을 둔 분류방법 <br>
일반적인 통계적 학습방법에서의 경험적 리스크 최소화와는 다른 구조적 리스트 최소화를 통해 오류를 최소화 시키는 방법 <br>
일반적 통계쩍 방법을 통한 합습에 실제 정확률은 학습에 이용된 데이터에 의해 제약을 받게 된다. <br>
학습자료로 주어진 n 차원에 벤터 공간에서 분류 공간간에 모든 점 사이의 거리를 최대화하도록 만들어 하나의 평명을 구해내는 것을 목표로 한다. 

# 데이터 로딩 (Iris)

In [6]:
setwd("/home/dev/R_study_jupyter/data")
iris = read.csv("iris.csv")
print(iris)

     ID Sepal_Length Sepal_Width Petal_Length Petal_Width    Species
1     1          5.1         3.5          1.4         0.2     setosa
2     2          4.9         3.0          1.4         0.2     setosa
3     3          4.7         3.2          1.3         0.2     setosa
4     4          4.6         3.1          1.5         0.2     setosa
5     5          5.0         3.6          1.4         0.2     setosa
6     6          5.4         3.9          1.7         0.4     setosa
7     7          4.6         3.4          1.4         0.3     setosa
8     8          5.0         3.4          1.5         0.2     setosa
9     9          4.4         2.9          1.4         0.2     setosa
10   10          4.9         3.1          1.5         0.1     setosa
11   11          5.4         3.7          1.5         0.2     setosa
12   12          4.8         3.4          1.6         0.2     setosa
13   13          4.8         3.0          1.4         0.1     setosa
14   14          4.3         3.0  

# 간단하게 SVM 모델 훈련 및 예측
SVM 을 사용하여 모델을 만들고 예측을 진행하여 보자 

In [10]:
#install.packages("kernlab")
library(kernlab)
(m<-ksvm(Species~Sepal_Length  + Sepal_Width + Petal_Length + Petal_Width , data=iris))
# 테스트 데이터 생성 
Sepal_Length = 5.0
Sepal_Width = 3.0
Petal_Length = 0.1
Petal_Width = 1.3
(test<-data.frame(Sepal_Length,Sepal_Width,Petal_Length,Petal_Width))
# 만들어진 모델로 예측 (virginica가 나왔네?)
head(predict(m, newdata=test))

Support Vector Machine object of class "ksvm" 

SV type: C-svc  (classification) 
 parameter : cost C = 1 

Gaussian Radial Basis kernel function. 
 Hyperparameter : sigma =  0.781477816576304 

Number of Support Vectors : 59 

Objective Function Value : -4.4757 -4.9713 -20.3593 
Training error : 0.026667 

Sepal_Length,Sepal_Width,Petal_Length,Petal_Width
5,3,0.1,1.3


[1] virginica
Levels: setosa versicolor virginica

# 단항, 다항 커널
단항 커널은 하나의 직선으로 그룹을 구분하는 것으로 어떻게 보면 lda 와 유사하다고도 할 수 있겠다. 다항 커널은 여러개의 직선으로 그룹을 구분하는 것으로 더 비선형적인 데이터를 구분을 잘 할 수 있게 된다. 하지만, 지나치게 구분할 경우 Overfitting 의 함정의 빠질 수 있으니 조심하여야 할 듯 함. 

In [22]:
test
m2 = ksvm(Species~Sepal_Length  + Sepal_Width + Petal_Length + Petal_Width, data=iris, kernel="vanilladot")
predict(m2, newdata=test)
m3 = ksvm(Species~Sepal_Length  + Sepal_Width + Petal_Length + Petal_Width, data=iris, kernel="polydot", kpar=list(degree=3))
predict(m3, newdata=test)

Sepal_Length,Sepal_Width,Petal_Length,Petal_Width
5,3,0.1,1.3


 Setting default kernel parameters  


[1] setosa
Levels: setosa versicolor virginica

[1] setosa
Levels: setosa versicolor virginica

# 튜닝?
음...

In [12]:
#install.packages("e1071")
library(e1071)
result <- tune(svm, Species ~., data=iris, gamma=2^(-1:1), cost=2^(2:4))
attributes(result)
result$best.parameters["dummyparameter"]

$names
[1] "best.parameters"  "best.performance" "method"           "nparcomb"        
[5] "train.ind"        "sampling"         "performances"     "best.model"      

$class
[1] "tune"

dummyparameter
0


# 앙상블 
여러개의 모델을 복합적으로 사용하는 것인데 그 방법으로는 3가지 정도가 있을 수 있다. <br>
Bagging.<br> Building multiple models (typically of the same type) from different subsamples of the training dataset.<br>
Boosting.<br> Building multiple models (typically of the same type) each of which learns to fix the prediction errors of a prior model in the chain.<br>
Stacking.<br> Building multiple models (typically of differing types) and supervisor model that learns how to best combine the predictions of the primary models.<br>